## LDA: 문서 내에서 숨겨진 주제 추출


In [1]:
!pip install numpy==1.23.5
!pip install scipy==1.10.1
!pip install gensim==4.3.1

  Using cached scipy-1.10.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (58 kB)
Using cached scipy-1.10.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (34.1 MB)
  Attempting uninstall: scipy
    Found existing installation: scipy 1.13.1
    Uninstalling scipy-1.13.1:
      Successfully uninstalled scipy-1.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imbalanced-learn 0.13.0 requires numpy<3,>=1.24.3, but you have numpy 1.23.5 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.23.5 which is incompatible.
jaxlib 0.5.1 requires scipy>=1.11.1, but you have scipy 1.10.1 which is incompatible.
pymc 5.22.0 requires numpy>=1.25.0, but you have numpy 1.23.5 which is incompatible.
scikit-image 0.25.2 requires numpy>=1.24, but you have numpy 1.23.5 which is incompatible.
scikit-image 0.25.2 requires 

In [3]:
import requests
import lxml.html
import pandas
import warnings
import time

In [4]:
# KoNLPy는 파이썬 기반의 한국어 자연어 처리(NLP) 라이브러리, 한국어 텍스트를 분석하고 처리하는 데 유용한 다양한 기능을 제공
!pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 58.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 28.9 MB/s eta 0:00:00


In [5]:
# NLTK 텍스트 분석 및 언어 데이터 처리에 널리 사용
!pip install nltk

In [6]:
s='한글의 형태소분석 결과를 확인하는 예제문장입니다. ①대한민국은 민주공화국이다. ②대한민국의 주권은 국민에게 있고, 모든 권력은 국민으로부터 나온다.'

In [8]:
# 형태소 분석을 위한 코드
import konlpy
from konlpy.tag import Okt # 형태소 분석기

okt = Okt()
print(okt.tagset)
okt.morphs(s) # morphs 는 해당 문자를 형태소 단위로 나누어 리스트로 변환

{'Adjective': '형용사', 'Adverb': '부사', 'Alpha': '알파벳', 'Conjunction': '접속사', 'Determiner': '관형사', 'Eomi': '어미', 'Exclamation': '감탄사', 'Foreign': '외국어, 한자 및 기타기호', 'Hashtag': '트위터 해쉬태그', 'Josa': '조사', 'KoreanParticle': '(ex: ㅋㅋ)', 'Noun': '명사', 'Number': '숫자', 'PreEomi': '선어말어미', 'Punctuation': '구두점', 'ScreenName': '트위터 아이디', 'Suffix': '접미사', 'Unknown': '미등록어', 'Verb': '동사'}


['한글',
 '의',
 '형태소',
 '분석',
 '결과',
 '를',
 '확인',
 '하는',
 '예',
 '제',
 '문장',
 '입니다',
 '.',
 '①',
 '대한민국',
 '은',
 '민주공화국',
 '이다',
 '.',
 '②',
 '대한민국',
 '의',
 '주권',
 '은',
 '국민',
 '에게',
 '있고',
 ',',
 '모든',
 '권력',
 '은',
 '국민',
 '으로부터',
 '나온다',
 '.']

In [10]:
malist = okt.pos(s, norm=True, stem=True)
print(malist)

[('한글', 'Noun'), ('의', 'Josa'), ('형태소', 'Noun'), ('분석', 'Noun'), ('결과', 'Noun'), ('를', 'Josa'), ('확인', 'Noun'), ('하다', 'Verb'), ('예', 'Modifier'), ('제', 'Modifier'), ('문장', 'Noun'), ('이다', 'Adjective'), ('.', 'Punctuation'), ('①', 'Foreign'), ('대한민국', 'Noun'), ('은', 'Josa'), ('민주공화국', 'Noun'), ('이다', 'Josa'), ('.', 'Punctuation'), ('②', 'Foreign'), ('대한민국', 'Noun'), ('의', 'Josa'), ('주권', 'Noun'), ('은', 'Josa'), ('국민', 'Noun'), ('에게', 'Josa'), ('있다', 'Adjective'), (',', 'Punctuation'), ('모든', 'Noun'), ('권력', 'Noun'), ('은', 'Josa'), ('국민', 'Noun'), ('으로부터', 'Josa'), ('나오다', 'Verb'), ('.', 'Punctuation')]


In [12]:
from google.colab import drive
drive.mount('/content/drive')

!cp "/content/drive/MyDrive/ai텍스트 분석/토픽 모델링/04_test.csv" "/content/"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
cp: cannot stat '/content/drive/MyDrive/ai텍스트 분석/토픽 모델링/04_test.csv': No such file or directory


In [14]:
!cp "/content/drive/MyDrive/ai텍스트 분석/토픽 모델링/04_text.csv" "/content/"

In [16]:
import pandas as pd
import numpy as np

df = pd.read_csv('/content/04_text.csv')
df.shape

(131, 3)

In [17]:
df.head()

,구분,조번호,내용
0,전문,전문,유구한 역사와 전통에 빛나는 우리 대한국민은 3·1운동으로 건립된 대한민국임시정부의...
1,총강,제1조,"①대한민국은 민주공화국이다. ②대한민국의 주권은 국민에게 있고, 모든 권력은 국민으..."
2,총강,제2조,①대한민국의 국민이 되는 요건은 법률로 정한다. ②국가는 법률이 정하는 바에 의하여...
3,총강,제3조,대한민국의 영토는 한반도와 그 부속도서로 한다.
4,총강,제4조,"대한민국은 통일을 지향하며, 자유민주적 기본질서에 입각한 평화적 통일 정책을 수립하..."


In [18]:
# 영어의 경우 소문자로 안하면 CountVectorizer 에서 warning 발생
# 금칙어(stopwords) 설정 // 아무 값도 안 줘도 됨. 이렇게 --> stopwords=[]
stopwords=[]
print(stopwords)

[]


In [20]:
# 원천 데이터가 형테소 분석이 안된 상태일 때 사용(명사만 추출)
def get_noun_list(text):
  words = []
  malist = okt.pos(text)
  for word in malist:
    if word[1] == "Noun":  # 명사 확인
        w = word[0]
        if len(w) >1 and w not in stopwords:
          words.append(w)

  return words

In [21]:
# 위에 업로드한 파일에서 '내용' 컬럼을 분석 대상 설정
constitution = []
constitution = df['내용'].astype(str)

In [22]:
#Scikit-learn 라이브러리에서 제공하는 자연어 처리 도구로, 텍스트 데이터를 Bag of Words (BoW) 방식으로 변환하는 역할을 합니다.
from sklearn.feature_extraction.text import CountVectorizer

# 빈도 기반의 벡터를 만드는 작업 별 의미는 없음. DTM 만드는 작업 중 일부
cv = CountVectorizer(tokenizer=get_noun_list, max_features=1000, stop_words = stopwords)
dtm = cv.fit_transform(constitution)
words = cv.get_feature_names_out()

# dtm이란 문서 내 단어들의 출현 빈도를 기반으로 수치화하여 머신러닝 모델에서 활용할 수 있도록 변환하는 과정

/usr/local/lib/python3.11/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [24]:
dtm.shape

(131, 830)

In [45]:
import gensim
#문서에서 의미 있는 정보를 추출하고, 텍스트 데이터를 벡터화하는 기능을 제공

from gensim.matutils import Sparse2Corpus
# 희소 행렬(Sparse Matrix)을 Gensim의 corpus 형식으로 변환하는 함수

corpus = Sparse2Corpus(dtm.T)

In [30]:
from gensim.corpora.dictionary import Dictionary

dictionary = Dictionary.from_corpus(
    corpus,
    id2word = dict(enumerate(words))
)

In [34]:
# topic 수를 찾기 위한 coherence 값 출력
# perplexity와 coherence란 값을 계산하는 건데, perplexity는 값이 작을 수록, coherence는 클 수록 좋음
# 토픽수를 순차적으로 바꾸면서 토픽 내부를 구성 하는 단어들의 유사도를 계산하는 방식(cohorence)
# 참고로 perplexity나 coherence 값이 예쁘다 하여 의미적으로 완전한 토픽이라고 볼 수는 없음.
# https://coredottoday.github.io/2018/09/17/%EB%AA%A8%EB%8D%B8-%ED%8C%8C%EB%9D%BC%EB%AF%B8%ED%84%B0-%ED%8A%9C%EB%8B%9D/ 이 사이트를 참고.

from gensim import corpora, models, similarities
from gensim.models.coherencemodel import CoherenceModel

coherencesT=[]
perplexitiesT=[]
warnings.filterwarnings('ignore')

# topic 수를 3~21 까지 돌리면서 coherence 값 출력
topic_range = range(3,21)

for i in  topic_range:
  nwords =100
  ntopics = i

  tic = time.time()
  lda4 = models.ldamodel.LdaModel(corpus, id2word=dictionary, num_topics=ntopics, iterations = 50, passes=15)
  print('ntopics LdaModel: ',ntopics,time.time() - tic)

  tic =time.time()
  cm = CoherenceModel(model=lda4, corpus=corpus, coherence='u_mass')
  coherence = cm.get_coherence()
  print("Coherence", coherence)
  coherencesT.append(coherence)

  log_perplexity_val = lda4.log_perplexity(corpus)
  print('Perplexity: ', log_perplexity_val)
  perplexitiesT.append(log_perplexity_val)

  print("Coherence, Perplexity elaspsed time: ", ntopics,time.time() - tic, '\n\n' )



ntopics LdaModel:  3 2.524188756942749
Coherence -6.305959992388932
Perplexity:  -6.2593784939414
Coherence, Perplexity elaspsed time:  3 0.05232405662536621 


ntopics LdaModel:  4 1.3792164325714111
Coherence -8.04673153998128
Perplexity:  -6.30228672766786
Coherence, Perplexity elaspsed time:  4 0.09632039070129395 


ntopics LdaModel:  5 3.0578196048736572
Coherence -7.251656851667162
Perplexity:  -6.351053312127845
Coherence, Perplexity elaspsed time:  5 0.10168194770812988 


ntopics LdaModel:  6 2.0133109092712402
Coherence -9.223534194320182
Perplexity:  -6.395798497613561
Coherence, Perplexity elaspsed time:  6 0.04666471481323242 


ntopics LdaModel:  7 1.148848056793213
Coherence -7.8438355339573125
Perplexity:  -6.376525553215421
Coherence, Perplexity elaspsed time:  7 0.05374717712402344 


ntopics LdaModel:  8 1.0860874652862549
Coherence -8.960003812359584
Perplexity:  -6.395195603726453
Coherence, Perplexity elaspsed time:  8 0.0476841926574707 


ntopics LdaModel:  9 1

## LDA 계산 부분


In [35]:
from gensim.models.ldamodel import LdaModel

# 토픽수를 임의로 9 로 설정
# coherence 나 perplexity를 계산하여 토픽 수를 지정할 수 있는 방법도 있음

topic_k = 10
lda = LdaModel(corpus=corpus, id2word=dict(enumerate(words)),
               num_topics = topic_k, passes =15, iterations = 50,
               random_state=1234)


In [36]:
# 0번 토픽 출력
lda.show_topic(0)

[('국민', 0.048052844),
 ('법률', 0.027414976),
 ('자유', 0.026659422),
 ('모든', 0.023583824),
 ('권리', 0.021183323),
 ('경제', 0.015460905),
 ('구속', 0.01545981),
 ('경우', 0.013558746),
 ('체포', 0.013552787),
 ('공개', 0.013550796)]

In [38]:
def dtm2doc(dtm, i):
  doc = []
  for i, n in enumerate(dtm[i].toarray().flat):
    if n > 0:
      doc.append((i,n))
  return doc

doc = dtm2doc(dtm, 35)

In [40]:
lda.get_document_topics(doc)

[(7, 0.96249455)]

In [41]:
# 뒤에 버전을 명시하지 않으면, 최신 버전 3.x 를 설치하는데, 이럴 경우 설치 셀 아래의 LDAvis 실행 코드에서 문법 에러 남.
#!pip install pyLDAvis==2.1.2  # 에러가 발생하기 때문에 실행하지 않습니다

In [42]:
pip install --upgrade pyLDAvis #수정하여 최신 함수로 설치 실행합니다.

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 2.7 MB/s eta 0:00:00
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 32.6 MB/s eta 0:00:00
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jaxlib 0.5.1 requires scipy>=1.11.1, but you have scipy 1.10.1 which is incompatible.
scikit-image 0.25.2 requires scipy>=1.11.4, but you have scipy 1.10.1 which is incompatible.
jax 0.5.2 requires scipy>=1.11.1, but you have scipy 1.10.1 which is incompatible.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >=

In [43]:
import pyLDAvis
import pyLDAvis.gensim as gensimvis

prepared_data = gensimvis.prepare(lda, corpus, dictionary)
pyLDAvis.display(prepared_data)

## LSI

In [47]:
constitution[0]

'유구한 역사와 전통에 빛나는 우리 대한국민은 3·1운동으로 건립된 대한민국임시정부의 법통과 불의에 항거한 4·19민주이념을 계승하고, 조국의 민주개혁과 평화적 통일의 사명에 입각하여 정의·인도와 동포애로써 민족의 단결을 공고히 하고, 모든 사회적 폐습과 불의를 타파하며, 자율과 조화를 바탕으로 자유민주적 기본질서를 더욱 확고히 하여 정치·경제·사회·문화의 모든 영역에 있어서 각인의 기회를 균등히 하고, 능력을 최고도로 발휘하게 하며, 자유와 권리에 따르는 책임과 의무를 완수하게 하여, 안으로는 국민생활의 균등한 향상을 기하고 밖으로는 항구적인 세계평화와 인류공영에 이바지함으로써 우리들과 우리들의 자손의 안전과 자유와 행복을 영원히 확보할 것을 다짐하면서 1948년 7월 12일에 제정되고 8차에 걸쳐 개정된 헌법을 이제 국회의 의결을 거쳐 국민투표에 의하여 개정한다.'

In [48]:
# 역토큰화 (토큰화 작업을 역으로 되돌림)
detokenized_doc = []

for i in range(len(constitution)):
  t = ' '.join(get_noun_list(constitution[i]))
  detokenized_doc.append(t)

detokenized_doc[0]

'유구 역사 전통 우리 국민 운동 건립 대한민국 임시정부 통과 불의 항거 민주 이념 계승 조국 민주 개혁 평화 통일 사명 각하 정의 인도 동포 로써 민족 단결 공고 모든 사회 폐습 불의 타파 자율 조화 바탕 자유민주 질서 더욱 정치 경제 사회 문화 모든 영역 각인 기회 능력 최고 도로 발휘 자유 권리 책임 의무 완수 국민 생활 향상 기하 항구 세계 평화 인류 공영 이바지 우리 우리 자손 안전 자유 행복 확보 다짐 제정 개정 헌법 이제 국회 의결 국민투표 개정'

In [49]:
df['내용'][0]

'유구한 역사와 전통에 빛나는 우리 대한국민은 3·1운동으로 건립된 대한민국임시정부의 법통과 불의에 항거한 4·19민주이념을 계승하고, 조국의 민주개혁과 평화적 통일의 사명에 입각하여 정의·인도와 동포애로써 민족의 단결을 공고히 하고, 모든 사회적 폐습과 불의를 타파하며, 자율과 조화를 바탕으로 자유민주적 기본질서를 더욱 확고히 하여 정치·경제·사회·문화의 모든 영역에 있어서 각인의 기회를 균등히 하고, 능력을 최고도로 발휘하게 하며, 자유와 권리에 따르는 책임과 의무를 완수하게 하여, 안으로는 국민생활의 균등한 향상을 기하고 밖으로는 항구적인 세계평화와 인류공영에 이바지함으로써 우리들과 우리들의 자손의 안전과 자유와 행복을 영원히 확보할 것을 다짐하면서 1948년 7월 12일에 제정되고 8차에 걸쳐 개정된 헌법을 이제 국회의 의결을 거쳐 국민투표에 의하여 개정한다.'

In [52]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_v = TfidfVectorizer(stop_words='english', max_features= 1000,
                          # 상위 1,000개의 단어를 보존 ## tokenizer=get_noun_list,
                          max_df = 0.5,
                          smooth_idf=True)

X = tfidf_v.fit_transform(detokenized_doc)
X.shape # TF-IDF 행렬의 크기 확인

(131, 829)

In [55]:
from sklearn.decomposition import TruncatedSVD


In [56]:
#svd_model = TruncatedSVD(n_components=10, algorithm='randomized', n_iter=10, random_state=1234)
# n_iter = 1 일때는 세션 다운로드가 안되지만, 5 로만 설정해도 세션 다운됨.
svd_model = TruncatedSVD(n_components=10, algorithm='randomized', n_iter=2, random_state=1234)
svd_model.fit_transform(X)
len(svd_model.components_)

10

In [57]:
np.shape(svd_model.components_)

(10, 829)

In [58]:
#terms = tfidf_v.get_feature_names() # 단어 집합. 829개의 단어가 저장됨. # deprecated

# 수정
terms = tfidf_v.get_feature_names_out() # 단어 집합. 829개의 단어가 저장됨.


def get_topics(components, feature_names, n=10):
    for idx, topic in enumerate(components):
        print("Topic %d:" % (idx+1), [(feature_names[i], topic[i].round(5)) for i in topic.argsort()[:-n - 1:-1]])

get_topics(svd_model. components_, terms)

Topic 1: [('대통령', 0.38105), ('국민', 0.25407), ('국회', 0.24578), ('국가', 0.24549), ('모든', 0.18093), ('국무총리', 0.16743), ('국회의원', 0.15614), ('헌법', 0.15097), ('국무위원', 0.14436), ('의원', 0.13266)]
Topic 2: [('국민', 0.44921), ('모든', 0.41999), ('자유', 0.23994), ('국가', 0.17634), ('의무', 0.14635), ('권리', 0.13051), ('침해', 0.12456), ('보호', 0.10191), ('비밀', 0.10125), ('노력', 0.09425)]
Topic 3: [('국회의원', 0.6596), ('임기', 0.19699), ('국회', 0.19457), ('제출', 0.17623), ('정부', 0.12679), ('의결', 0.09711), ('선거', 0.07524), ('체포', 0.07244), ('구금', 0.0638), ('예산', 0.06248)]
Topic 4: [('국회의원', 0.39289), ('국무총리', 0.29541), ('국무위원', 0.26067), ('행정각부', 0.14465), ('대통령', 0.12793), ('임명', 0.10327), ('제출', 0.09363), ('모든', 0.08393), ('정부', 0.08312), ('국무회의', 0.08288)]
Topic 5: [('국가', 0.3305), ('정책', 0.16326), ('회의', 0.13469), ('경제', 0.12139), ('기타', 0.11736), ('자문', 0.11436), ('사항', 0.11297), ('정부', 0.09738), ('국회의원', 0.09648), ('노력', 0.09524)]
Topic 6: [('임기', 0.33007), ('대법원', 0.24911), ('대법관', 0.21382), ('국회의원', 0.18662),